In [2]:
'''The below cell was used to load some gitlab  repos. It was obviously based on the
   sample code on the class repo. I have commented out the portion that actually does
   the loading so I won't just load mutiple version of the same repos documents after
   doing so many times before I realized what I was actually doing. If you want you can
   uncomment this portion to check that it works. I also added a line to clear out my 
   collection. I did this after I realized what I had done when I checked the number of
   documents in my collection and found that there were over 30000. I then commented 
   this out and ran the get method again to reload the collection. I then commented that
   out as stated above.
   Other relevant notes:
       line 24: This is were I cleared out my collection so I could start from scratch
               ----Currently commented out
       line 107: This is were I load the gitlab documents representing projects
               ----Currently commented out
'''

#the below code was used to load a bunch of the projects from git lab
import sys
import re
import pymongo
import json
import time
import datetime
import requests
import pprint


dbname = "fdac18mp2" #please use this database
collname = "glprj_gjones2" #please modify so you store data in your collection
# beginning page index
begin = "0"

# start pymongo client
client = pymongo.MongoClient()

db = client[dbname]

coll = db[collname]

#coll.delete_many({})

print(coll.count_documents({}))

beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

gleft = 0

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left


# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                coll.insert_one(el)
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            coll.insert_one(el)
                    else:
                        sys.stderr.write("url can not found2:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found1:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')

#start retrieving        
#get(beginurl,coll)        

792


In [60]:
'''The below cell at one point looked through the loaded gitlab project documents and found all of those
   projects that have a name starting with n. It then removed all other projects. I later added stuff to give read
   outs of the total counts of all documents, source forge documents, and gitlab documents
   Notes:
           --The line that actually did the removal was (line 68) now commented our so its safe to run 
           --I modified this later so the added source forge repos would be preserved, but at first these would have been
             removed
           --I once did this with a for loop (line 55) but realized I could accomplish the task 
             with pymongo function delete many and the comparison query operator $in and a list of the names to keep
           --The line 27 created a text index for http_url_to_repo so I could search for specific types of url's
             such as gitlab and sourceforge. After I did the adjustments to the collection I added the lines(38, 39)
             that show how many different urls of the two types exits in my collection
           --As it is now it will tell you total counts of all documents in collection, source forge documents, 
             and gitlab documents
'''
import pymongo


dbname = "fdac18mp2" #please use this database
collname = "glprj_gjones2" #please modify so you store data in your collection

# start pymongo client
client = pymongo.MongoClient()

db = client[dbname]

coll = db[collname]

# http_url_to_repo

# create an index so we can search for url key words
#coll.create_index( [( 'http_url_to_repo', "text") ] )

print('There are {:d} total documents in the collection {:s}'.format(coll.count_documents({}), collname))

# grab a cursor to all of the projects
#all_projects = coll.find({})

# grab a cursor to all projects that start with n
n_projects = coll.find({'name': {'$regex': '^n'}})
# grabing both gitlab and source forge documents

gitlab_projects = coll.find({'$text': {'$search': 'gitlab'}})
source_forge_projects = coll.find({'$text': {'$search': 'sourceforge'}})
      
print('there are {:d} source forge projects'.format(coll.count_documents({'$text': {'$search': 'sourceforge'}})))
print('there are {:d} gitlab projects'.format(coll.count_documents({'$text': {'$search': 'gitlab'}})))
print('there are {:d} projects that start with n'.format(coll.count_documents({'name': {'$regex': '^n'}})))

to_keep_array = list()

# load the names of git lab projects starting with n
for project in n_projects:
    to_keep_array.append(project['name'])

#for project in source_forge_projects:
    #to_keep_array.append(project['name'])

    
    
# used to count how many projects I have that start with n
print("the length of the to keep array is {:d}".format(len(to_keep_array)))
   

    
# delete all project documents that have names 
# not in the to keep array
#coll.delete_many({'name': {'$nin': to_keep_array}})    
    
    
# This for loop 
#for project in all_projects:
#    name = project['name']
#    if name not in to_keep_array:
#        coll.delete_one({'name': name})
   

#print('The number of documents in my collection is now {:d}'.format(coll.count_documents({})))



There are 860 total documents in the collection glprj_gjones2
there are 68 source forge projects
there are 792 gitlab projects
there are 860 projects that start with n
the length of the to keep array is 860


In [16]:
'''The below cell was used to load some source forge  repos.
   Other relevant code and Notes:
       url_tester: (line 137 method call) The call to this  method was used to test the urls of projects to make sure they 
                  still existed
                   --The call to this code is currently commented out since I already did the tests
       get_projects_dict_ffile: (line 95 declaration, 131 method call) creates a list dictionarys from a given file that 
                                contains urls to source forge project repos added by hand. The dictionary is of the form:
                                key=name, val = name of repo
                                key=http_url_to_repo, val = url to repo
        --line 155: adds the the list of dictionaries to my collection. This line is commented out due to me already running
                   this once. 
'''
import sys
import re


import pymongo
import json
import time
import datetime
import requests
import pprint


dbname = "fdac18mp2" #please use this database
collname = "glprj_gjones2" #please modify so you store data in your collection

my_letter = 'n'

# start pymongo client
client = pymongo.MongoClient()

# grab reference to the class database
db = client[dbname]

# grab a reference to my collection
coll = db[collname]

# check how many documents are in my collection
# old_col_count = coll.count_documents({})
# print('The original count is {:d}'.format(old_col_count))

# will test the urls in d_list and make sure they go 
# somewhere. If the url doesn't it will be removed
def url_tester(d_list):
    cnt = 0
    bad_cnt = 0
    for dictL in d_list:
        url = dictL['http_url_to_repo']
        name = dictL['name']
        r = requests.get(url)
        print(r.status_code)
        if not r.ok or r.status_code != 200:
            print('There is a bad url for project {:s} at url {:s}'.format(name, url))
            #TODO: remove bad urls
        elif r.status_code == 403:
            print('we got blocked!!!!')
        cnt +=1
        if cnt == 100: 
            break
    return bad_cnt, cnt


# grabs the name from a url to a project
# and returns it.
# used for the source forge urls to get the name of the repo
def get_name(url):
    stp = len(url)-1
    ps =url.find('projects', 0)
    fs = url.find('/' ,ps)
    strt = fs + 1
    return url[strt:stp]


# creates a dictionary out of the url
# argument were the keys are:
# name = name of the project
# http_url_to_repo = the url to the repo named 'name'
# and returns that dictionary
# will be used to create a list of dictionary's that 
# will be added to my collection
def add_projects(url):
    dict = {}
    # use the get_name method to get the name of the repo
    name = get_name(url)
    
    # make sure the project actually starts with the correct letter
    if name[0] == 'n':
        dict['name'] = name
        dict['http_url_to_repo']=url
    return dict

# This is what loads the sorce forge projects.
# opens and reads the source forge 
# urls stored in the file named SourceForge_urls.txt
# and returns a dictionary list where each entry in 
# the list is is of the form:
#                           name: name of project
#                           http_url_to_repo: the url to the repo of the given name
# also returns a list of the names of the projects added for debugging
def get_projects_dict_ffile(filename, c):

    f = open(filename, 'r')


    d_list = list()
    names = list()
    for line in f:
        
        # create a dictionary where the keys are the tags:
        #                                                name: name of project
        #                                                http_url_to_repo: the url to the project
        # see definition of methond above for more details
        n_dict = add_projects(line.strip('\n'))
        
        #store the names of the projects
        names.append(dict['name'])
    
        # if add_projects didn't find a project that starts with the
        # correct letter it returns a empty dictionary
        # if this happens do not add it but alert the user
        if len(n_dict) == 0:
            print('bad project name at {:d}'.format(c))
        else:
            d_list.append(n_dict)
        c += 1
    
    return d_list, names

# cnt = 1

# create a list containing dictionaries representing
# the projects and thier urls found in the named file
dlist, name_list = get_projects_dict_ffile('SourceForge_urls.txt', 1)



# used for debugging and testing
#for dict in dlist:
#    print(cnt,dict)
#    cnt +=1


# The below code test the urls loaded to make sure they exist.
# test the urls in the dictionary list(dlist) and make sure they still exist 
# if they do not they are removed
# url_tester(dlist)    


print('The number of names is {:d}'.format(len(name_list)))
print('There are {:d} entries in dlist'.format(len(dlist)))

# --------------------------------------------------------------Insert the source forge projects
#coll.insert_many(dlist)

#new_col_count = coll.count_documents({})
#print('The new count is {:d}'.format(new_col_count))

# my_add_projects = coll.find({name:{'$s': name_list} })

#cntt = 1
#for project in coll.find({'name':{'$in': name_list} }):
    
    #print('Name {:d}: {:s}'.format(cntt, project['name']))
    #cntt += 1
    
#for line in lines:
#    print(cnt,line)
#    cnt += 1

The original count is 860
The dictionary list contains 68
The number of names is 68
There are 68 entries in dlist
The new count is 860
Name 1: n-ctrl
Name 2: naps
Name 3: narxsim
Name 4: nas4free
Name 5: nasi
Name 6: ncron
Name 7: ndnbackpropneur
Name 8: ndata
Name 9: net-simulator
Name 10: netvalidator
Name 11: neur-php
Name 12: nfogen
Name 13: ngsmleditor
Name 14: ninu
Name 15: njmvcopensource
Name 16: njpc
Name 17: nlview
Name 18: nlingo
Name 19: nmea
Name 20: noperation
Name 21: nrrd-cpp
Name 22: ns2miraclewimax
Name 23: nsisdevprojects
Name 24: nwe00xmp3man
Name 25: nwn2ata
Name 26: nxwtools
Name 27: namin
Name 28: naivete
Name 29: namaless
Name 30: napatacms
Name 31: narga
Name 32: nativecppnetlibrary
Name 33: nativelibs4java
Name 34: nativeviewer
Name 35: nside
Name 36: nekit
Name 37: neotracker-time
Name 38: neoshark-2013
Name 39: neptunekrn
Name 40: neriaxi
Name 41: net4j
Name 42: netcvslib
Name 43: netdj
Name 44: netkit-srl
Name 45: nettext
Name 46: netbritelibrary
Name 47: n

In [16]:
'''This cell was used to test the gitlab urls to make sure they worked if they didn't they would be removed.
   I repeated this process until i got no bad links back 
   Notes:
         ---The line(32) that once deleted documents that didn't work has been commented out so if you want to test a url
             just uncomment it
'''

# now remove non working links

def gitlab_url_tester(collection, limit):

    cnt = 0
    bad_cnt = 0
    to_keep_names = list()

    for project in collection.find():
        url = project['http_url_to_repo']
        name = project['name']
        r = requests.get(url)
        #print(r.status_code)
        if not r.ok or r.status_code != 200:
            print('There is a bad url for project {:s} at url {:s}'.format(name, url))
            bad_cnt += 1
        elif r.status_code == 403:
            print('we got blocked!!!!')
        else:
            to_keep_names.append(name)
        cnt +=1
        if cnt == limit: 
            break
            
    #coll.delete_many({'name': {'$nin': to_keep_names}}) 
    
    return bad_cnt, cnt
limit = coll.count_documents({})

# Test the urls in collection
bad, cnt = gitlab_url_tester(coll, limit)

print('there are {:d} bad links in the first {:d}'.format(bad, cnt))
    
    
    

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [58]:
'''    This cell of code will allow the user to look at the different project names and url
       stored in my collection. It prompts the user to give a numeric choice of 0(source forge)
       or 1(gitlab) and the prompts the user to enter how many of that type they would like to see.
       This was mainly used for debugging and messing around with pymongo
'''
import pymongo


dbname = "fdac18mp2" #please use this database
collname = "glprj_gjones2" #please modify so you store data in your collection


# start pymongo client
client = pymongo.MongoClient()

options = ['0', '1']
p_types = ['source forge projects', 'git lab projects']

project_type = ''

while project_type != '0' and project_type != '1':
    print('Give me which type of projects you want to see:')
    print('0) source forge projects')
    print('1) git lab projects')
    project_type = input("What is your choice? ")
    print('you chose {:s}'.format(p_types[int(project_type)]))


    
limit = 0

choice = '-1'

if int(project_type) == 0:
    projects = coll.find({'$text': {'$search': 'sourceforge'}})
    limit = coll.count_documents({'$text': {'$search': 'sourceforge'}})
    
    while int(choice) <= 0 and int(choice) < limit:
    #while int(choice) not in options:
        choice = input('1How many projects would you like to see. Must be between 1 and {:d}'.format(limit))     
else:
    projects = coll.find({'$text': {'$search': 'gitlab'}})
    limit = coll.count_documents({'$text': {'$search': 'gitlab'}})
    
    while int(choice) <= 0 and int(choice) < limit:
        choice = input('How many projects would you like to see. Must be between 1 and {:d}: '.format(limit))

print('you chose  to see {:d} {:s}'.format(int(choice), p_types[int(project_type)]))
print('')      

cnt = 1
for project in projects:
    print('{:d}) name: {:s}, url: {:s}'.format(cnt, project['name'], project['http_url_to_repo']))
    cnt += 1
    if cnt > int(choice):
        break
    
    

Give me which type of projects you want to see:
0) source forge projects
1) git lab projects
What is your choice? 0
you chose source forge projects
1How many projects would you like to see. Must be between 1 and 6815
you chose  to see 15 source forge projects

1) name: network-ip-tools, url: https://sourceforge.net/projects/network-ip-tools/
2) name: npp-graphbuilder-plugin, url: https://sourceforge.net/projects/npp-graphbuilder-plugin/
3) name: n-ctrl, url: https://sourceforge.net/projects/n-ctrl/
4) name: net-simulator, url: https://sourceforge.net/projects/net-simulator/
5) name: neur-php, url: https://sourceforge.net/projects/neur-php/
6) name: nrrd-cpp, url: https://sourceforge.net/projects/nrrd-cpp/
7) name: neotracker-time, url: https://sourceforge.net/projects/neotracker-time/
8) name: neoshark-2013, url: https://sourceforge.net/projects/neoshark-2013/
9) name: netkit-srl, url: https://sourceforge.net/projects/netkit-srl/
10) name: nexuiz-extra, url: https://sourceforge.net/pro

In [8]:
print('The number of documents in my collection is:')
print(coll.count_documents({}))
print('the number of elements in to keep array is:')
print(len(to_keep_array))

The number of documents in my collection is:
792
the number of elements in to keep array is:
792
